# 3. Feature Engineering (Extracción de Características)

**Objetivo:** Transformar el texto crudo en un conjunto de métricas numéricas (Dataset Procesado).

**Estrategia:** Unificar el dataset Humano y el dataset AI en un solo DataFrame. Luego, aplicar técnicas de NLP para medir la riqueza de vocabulario, complejidad sintáctica, sentimiento y legibilidad.

El resultado será un archivo `.csv` donde cada fila es un artículo y cada columna es una métrica matemática, listo para entrenar el modelo XGBoost.

## 3.1. Configuración e importación de librerías

Importamos las herramientas necesarias. Usaremos `spaCy` como motor principal para el análisis sintáctico (gramática profunda) debido a su precisión y velocidad. También gestionamos la carga del modelo de idioma inglés (`en_core_web_sm`).

Importamos `pandas` para datos, `textblob` para análisis de sentimiento, `textstat` para métricas de legibilidad y `nltk` para tokenización avanzada.

In [ ]:
#%pip install spacy
#!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 5.3 MB/s eta 0:00:03
     ---- ----------------------------------- 1.6/12.8 MB 4.0 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 3.5 MB/s eta 0:00:04
     ------- -------------------------------- 2.4/12.8 MB 3.1 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 2.7 MB/s eta 0:00:04
     --------- ------------------------------ 3.1/12.8 MB 2.5 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.5 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.5 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.5 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.5 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 2.5 MB/s eta 0

In [15]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import textstat
import spacy 
import os

# Cargamos el modelo de inglés de spaCy
# (Es mucho más rápido y preciso que NLTK para gramática)
try:
    nlp = spacy.load("en_core_web_sm")
    print("✅ Modelo spaCy cargado correctamente.")
except OSError:
    print("❌ ERROR: No tienes el modelo descargado.")
    print("Ejecuta en tu terminal: python -m spacy download en_core_web_sm")

# --- RUTAS ---
HUMAN_FILE = '../data/1_raw/all-the-news-5k-sample.csv'
AI_FILE = '../data/3_synthetic/ai_generated_gemini.csv'
OUTPUT_FILE = '../data/2_processed/training_data_final.csv'

✅ Modelo spaCy cargado correctamente.


## 3.2. Definición de la Función de Extracción (El Motor NLP)

Esta función es el núcleo del proyecto. Recibe un texto crudo y devuelve un diccionario con sus "signos vitales" lingüísticos:

1.  **Diversidad Léxica:** Relación entre palabras únicas y total de palabras (riqueza de vocabulario).
2.  **Densidad Gramatical:** Proporción de palabras de contenido (verbos, sustantivos, adjetivos) frente a palabras funcionales.
3.  **Ratio de Adjetivos:** Las IAs a veces tienden a usar un lenguaje más "florido" o descriptivo.
4.  **Sentimiento:** Polaridad (positivo/negativo) y Subjetividad.
5.  **Legibilidad:** Nivel educativo necesario para entender el texto (Flesch-Kincaid).

In [21]:
def extract_features(text):
    """
    Extrae métricas avanzadas incluyendo Varianza y Estilo.
    """
    if not isinstance(text, str) or len(text.strip()) < 10:
        return None 
        
    # 1. Procesamiento spaCy
    doc = nlp(text)
    num_tokens = len(doc)
    if num_tokens == 0: return None
    
    # Lista de longitudes de cada frase (para calcular varianza)
    sentence_lengths = [len(sent) for sent in doc.sents]
    num_sentences = len(sentence_lengths)
    
    # 2. Métricas Gramaticales (POS Tagging)
    pos_counts = doc.count_by(spacy.attrs.POS)
    num_verbs = pos_counts.get(spacy.symbols.VERB, 0)
    num_adjs = pos_counts.get(spacy.symbols.ADJ, 0)
    num_nouns = pos_counts.get(spacy.symbols.NOUN, 0)
    num_nums = pos_counts.get(spacy.symbols.NUM, 0) # 🔥 NUEVO: Cifras/Números
    
    # 3. 🔥 NUEVO: Métricas de "Humanidad" (Caos y Contenido)
    
    # Stopwords (Palabras de relleno)
    num_stopwords = sum(1 for token in doc if token.is_stop)
    stopword_ratio = num_stopwords / num_tokens
    
    # Desviación Estándar de frases (Burstiness Proxy)
    # Si es 0 o bajo = Robot monótono. Si es alto = Humano caótico.
    sent_len_std = np.std(sentence_lengths) if num_sentences > 1 else 0
    
    # Ratio de Entidades Nombradas (Personas, Países, Empresas)
    # El periodismo humano suele tener muchas entidades específicas.
    num_entities = len(doc.ents)
    entity_ratio = num_entities / num_tokens
    
    # Riqueza léxica
    unique_words = len(set([token.text.lower() for token in doc if token.is_alpha]))
    lexical_diversity = unique_words / num_tokens if num_tokens > 0 else 0
    
    # 4. Sentimiento y Legibilidad
    blob = TextBlob(text)
    reading_ease = textstat.flesch_reading_ease(text)
    
    return {
        # --- Estructura ---
        'word_count': num_tokens,
        'avg_sentence_length': np.mean(sentence_lengths) if sentence_lengths else 0,
        'sent_len_std': sent_len_std,       # 🔥 CLAVE: Varianza de estructura
        
        # --- Estilo ---
        'lexical_diversity': lexical_diversity,
        'stopword_ratio': stopword_ratio,   # 🔥 CLAVE: "Fluff" ratio
        'adj_ratio': num_adjs / num_tokens,
        'verb_ratio': num_verbs / num_tokens,
        'noun_ratio': num_nouns / num_tokens, # 🔥 CLAVE: Densidad de información
        'entity_ratio': entity_ratio,         # 🔥 CLAVE: Anclaje en la realidad
        
        # --- Semántica ---
        'sentiment_polarity': blob.sentiment.polarity,
        'sentiment_subjectivity': blob.sentiment.subjectivity,
        'reading_ease': reading_ease
    }

## 3.3. Carga y Unificación de Datos

1.  Cargamos el CSV Humano y le asignamos la etiqueta `label = 0`.
2.  Cargamos el CSV de IA y le asignamos la etiqueta `label = 1`.
3.  Los fusionamos en un solo DataFrame grande (`df_full`).

In [ ]:
## 3.3. Carga y Unificación de Datos (Con Trazabilidad)
# ------------------------------------------
print("📂 Cargando datasets con trazabilidad...")

dfs = []

# 1. Cargar Humano
if os.path.exists(HUMAN_FILE):
    df_h = pd.read_csv(HUMAN_FILE)
    df_h['label'] = 0
    
    # CREAMOS EL ID: Usamos el índice como identificador único original
    # Esto nos permitirá decir: "Este texto humano es el padre del texto IA X"
    df_h['original_id'] = df_h.index
    
    # Seleccionamos columnas clave
    if 'content' in df_h.columns: df_h.rename(columns={'content': 'article'}, inplace=True)
    
    # Nos quedamos con ID, Texto y Etiqueta
    dfs.append(df_h[['original_id', 'article', 'label']].dropna())
    print(f"   -> Humanos: {len(df_h)} filas.")
else:
    print("❌ Falta archivo Humano.")

# 2. Cargar IA
if os.path.exists(AI_FILE):
    df_a = pd.read_csv(AI_FILE)
    df_a['label'] = 1
    
    # Aquí 'original_id' YA VIENE del Notebook 2 (¡tu gran acierto!)
    # Solo nos aseguramos de tener las mismas columnas
    dfs.append(df_a[['original_id', 'article', 'label']].dropna())
    print(f"   -> IA Generada: {len(df_a)} filas.")
else:
    print("⚠️ AVISO: Aún no existe el archivo de IA.")

# 3. Fusión Inteligente
if not dfs:
    print("🛑 No hay datos.")
else:
    df_full = pd.concat(dfs, ignore_index=True)
    
    # Verificación de integridad
    # Si el ID 5 existe en Humano pero no en IA (porque falló), no pasa nada.
    # Pero ahora tendremos el dato guardado.
    print(f"📊 Dataset Consolidado: {len(df_full)} filas.")
    print(f"   Ejemplo de trazabilidad (ID 0):")
    print(df_full[df_full['original_id'] == 0][['label', 'original_id']])

## 3.4. Ejecución del procesado y guardado

Iteramos sobre cada fila del dataset combinado, aplicamos la función `extract_features` y construimos el DataFrame final.
Este proceso puede tardar unos minutos dependiendo de la cantidad de textos.

In [ ]:
## 3.4. Procesado Masivo (Preservando IDs)
# ------------------------------------------
print("⚙️ Extrayendo características...")

features_list = []

for index, row in df_full.iterrows():
    metrics = extract_features(row['article'])
    
    if metrics:
        # 1. Guardamos la etiqueta (Target)
        metrics['label'] = row['label']
        
        # 2. Guardamos el ID ORIGINAL (Trazabilidad)
        metrics['original_id'] = row['original_id']
        
        features_list.append(metrics)
    
    if (index + 1) % 250 == 0:
        print(f"   ... {index + 1} procesados")

# Guardado Final
df_final = pd.DataFrame(features_list)

# REORDENAR COLUMNAS: Poner 'original_id' y 'label' al principio para que sea legible
cols = ['original_id', 'label'] + [c for c in df_final.columns if c not in ['original_id', 'label']]
df_final = df_final[cols]

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
df_final.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print(f"✅ ¡ÉXITO! Dataset final guardado en: {OUTPUT_FILE}")
print(df_final.head())